# 金融问答系统生成不准？端到端流程优化提升结果可靠性

## 1. 为什么我们需要更可靠的金融问答系统？


- 金融场景的特殊性：
  - 高专业性（ROE、EBITDA、非经损益）
  - 高时效性（年报数据不能错年）
  - 高风险性（一句错误可能引发合规问题）

- 大模型的“幻觉”现实：
  - “苹果2023年净利润是999亿美元” → 听起来合理，实为虚构
  - 传统 RAG 无法根治：检索→生成→输出，**缺乏验证闭环**

- 核心问题：
  > “我们能相信模型给出的答案吗？”





## 2. 传统 RAG 的局限性

| 传统 RAG 流程 | 问题 |
|---------------|------|
| 单次检索 → 单次生成 → 直接输出 |  无纠错机制 |
| 依赖模型“自觉”引用原文 |  幻觉无法拦截 |
| 流程不可见、不可控 |  难以调试与优化 |
| 无评估、无 trace |  无法持续迭代 |

> **RAG 不是终点，而是起点**


## 3. 我们的答案：将 AI 推理变为可编程工作流

- 核心理念：
  > **AI 推理 ≠ 黑盒输出，而应是可编程、可干预、可验证的决策流程**

- 类比：医生问诊流程
  - 问诊 → 检查 → 初步诊断 → 验证 → 复查
  - 我们的系统也应如此

- 技术选型：**LangGraph**
  - LangChain 推出的**状态化工作流引擎**
  - 支持条件跳转、循环、重试、状态管理
  - 让 AI 拥有“判断力”和“纠错能力”

## 4. 系统架构设计：六大模块闭环

```
用户提问
   ↓
[检索] → FAISS + Sentence Transformers（轻量本地检索）
   ↓
[生成] → GPT / 微调模型（模拟专业回答）
   ↓
[验证] → 内容一致性检查（防止幻觉）
   ↖______↓______↗
         |
   [LangGraph] ← 控制流程：错了吗？重试！
         ↓
   [评估] ← LangSmith（全流程 trace 与评估）
         ↓
   [服务] ← FastAPI（标准 API 接口）
```

## 5. 核心实现一：轻量级向量检索（FAISS）


- 不依赖远程向量库，本地部署即可运行
- 代码片段（高亮关键行）：

```python
embeddings = model.encode(texts)
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

# 检索
q_emb = model.encode([query])
_, indices = index.search(q_emb, 3)
```

- 优势：
  - 轻量、快速、支持中文
  - 可增量更新，适合企业私有化部署

## 6. 核心实现二：LangGraph 编排闭环流程

- 流程图：
  ```
  [检索] → [生成] → [验证]
                   ↓
              是？ → END
              否？ → 重试检索
  ```

- 关键代码：决策逻辑

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Dict
from retrieval.vector_store import VectorRetriever
from openai import OpenAI
import os

# 初始化组件
retriever = VectorRetriever("data/financial_docs.jsonl")
llm_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 定义工作流状态
class GraphState(TypedDict):
    question: str
    retrieved_docs: List[Dict]
    generation: str
    attempts: int
    verified: bool

# 节点1：检索
def retrieve(state: GraphState):
    docs = retriever.retrieve(state["question"])
    return {**state, "retrieved_docs": docs}

# 节点2：生成
def generate(state: GraphState):
    context = "\n".join([doc["text"] for doc in state["retrieved_docs"]])
    prompt = f"""
    请根据以下真实资料回答问题。若信息不足，请回答“无法确定”。
    资料：{context}
    问题：{state['question']}
    """
    response = llm_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return {**state, "generation": response.choices[0].message.content}

# 节点3：验证（防止幻觉）
def verify(state: GraphState):
    gen = state["generation"]
    # 检查生成答案是否包含任一原文片段
    matched = any(
        doc["text"].strip() in gen
        for doc in state["retrieved_docs"]
        if len(doc["text"].strip()) > 10
    )
    return {
        **state,
        "verified": matched,
        "attempts": state.get("attempts", 0) + 1
    }

# 决策函数：是否需要重试？
def should_retry(state: GraphState):
    if state["verified"] or state["attempts"] >= 2:
        return "end"
    return "retrieve"  # 可扩展为 rewrite 后重检
# 构建图
def create_rag_graph():
    workflow = StateGraph(GraphState)
    workflow.add_node("retrieve", retrieve)
    workflow.add_node("generate", generate)
    workflow.add_node("verify", verify)

    workflow.set_entry_point("retrieve")
    workflow.add_edge("retrieve", "generate")
    workflow.add_edge("generate", "verify")
    workflow.add_conditional_edges(
        "verify",
        should_retry,
        {"end": END, "retrieve": "retrieve"}
    )
    return workflow.compile()
```

- 优势：
  - 支持多跳推理
  - 可扩展为 query rewrite、多源检索等


## 7. 核心实现三：验证机制防止幻觉


- 验证逻辑：
  - 检查生成答案是否包含原文片段
  - 简单但有效，拦截 80%+ 明显幻觉

```python
matched = any(doc["text"].strip() in generation for doc in docs)
```

- 进阶方向：
  - NLI 模型判断蕴含关系
  - 数字提取比对（如净利润）
  - 用户反馈打标

## 8. 核心实现四：FastAPI 服务化部署

- 提供标准 RESTful 接口：

```bash
curl -X POST http://localhost:8000/qa \
     -H "Content-Type: application/json" \
     -d '{"question": "苹果公司2023年的净利润是多少？"}'
```

执行预测，返回预测结果
```python
# api/main.py
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse
from graph.rag_workflow import create_rag_graph
import asyncio

app = FastAPI(title="金融问答系统")
rag_app = create_rag_graph()

@app.post("/qa")
async def question_answering(request: Request):
    data = await request.json()
    question = data.get("question", "").strip()
    if not question:
        return JSONResponse({"error": "缺少问题"}, status_code=400)

    # 同步调用 LangGraph（生产环境建议异步）
    input_state = {"question": question, "attempts": 0, "verified": False}
    loop = asyncio.get_event_loop()
    result = await loop.run_in_executor(None, lambda: rag_app.invoke(input_state))

    return JSONResponse({
        "answer": result["generation"],
        "sources": [doc["meta"] for doc in result["retrieved_docs"]],
        "verified": result["verified"],
        "attempts": result["attempts"]
    })

@app.get("/")
def home():
    return {"message": "金融问答系统运行中", "endpoints": ["/qa (POST)"]}
```

- 返回结构化结果：

```json
{
  "answer": "998亿美元",
  "sources": [{"company": "Apple", "year": 2023, ...}],
  "verified": true
}
```

- 优势：易于集成到客服、投研、合规系统


## 9. 全流程可观测：LangSmith 监控与评估


- LangSmith 能做什么？
  - 自动记录每一次调用的完整 trace
  - 查看检索内容、prompt、生成结果
  - 分析耗时瓶颈（检索 vs 生成）
  - 添加自定义评估指标（FactScore、相关性）
  - A/B 测试不同策略

![langsmith1](24/image.png)

## 10. 工程化价值总结


| 维度 | 本方案优势 |
|------|-----------|
| **可靠性** | 验证+重试，显著降低幻觉 |
| **可解释性** | 答案来源可追溯 |
| **可控性** | 流程可编程、可干预 |
| **可评估性** | LangSmith 全链路监控 |
| **可部署性** | FastAPI 标准服务 |
| **可迭代性** | 支持反馈驱动优化 |

> 这不是一个 Demo，而是一个 **可交付的工程系统**

## 结论与展望


- **结论**：
  > 在金融AI中，**可信比智能更重要**。  
  > 可靠的系统，必须建立在**工程化、流程化、可评估**的基础之上。

- **未来方向**：
  - 加入 Query Rewrite 提升召回
  - 使用 LoRA 微调模型，降低成本依赖
  - 支持 PDF 表格解析（财报结构化）
  - 构建用户反馈闭环，驱动持续优化

- **一句话总结**：
  > **我们不追求“一次生成就正确”，而是追求“即使错了也能发现并纠正”。**
